# Fire response traits in plants from field samples

This script contains examples of R code to query tables in the database

## Load libraries

In [1]:
library(RPostgreSQL)
library(ggplot2)
library(dplyr)
require(tidyr)
library(galah)
library(ozmaps)
library(sf)
library(data.tree)

Loading required package: DBI


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: tidyr


Attaching package: ‘galah’


The following object is masked from ‘package:stats’:

    filter


Linking to GEOS 3.11.0, GDAL 3.5.3, PROJ 9.1.0; sf_use_s2() is TRUE



In [2]:
here::i_am("read-from-database/Read-table-traits-per-species.ipynb")

here() starts at /Users/z3529065/proyectos/fireveg/fireveg-analysis



In [3]:
galah_config(email = readLines(here::here("secrets","galah-email.txt")))

## Query taxonomy from Atlas of Living Australia

In [4]:
nsw <- st_transform(ozmaps::ozmap_states, 4326) %>% filter(NAME == "New South Wales")

In [5]:
plant_families <- galah_call() |>
  galah_identify("plantae") |>
galah_geolocate(nsw, type = "bbox")|>
  galah_down_to(family) |>
  atlas_taxonomy()

Data returned for bounding box:
xmin = 140.99927 xmax = 153.62986 ymin = -37.50503 ymax = -28.15703


In [6]:
print(plant_families, pruneMethod = "dist", limit = 10)

                           levelName
1  Plantae                          
2   ¦--Rhodoplantae                 
3   ¦   °--Glaucophyta              
4   ¦       °--... 1 nodes w/ 2 sub 
5   ¦--Anthocerotophyta             
6   ¦   °--Anthocerotopsida         
7   ¦       °--... 3 nodes w/ 6 sub 
8   ¦--Bryophyta                    
9   ¦   °--... 4 nodes w/ 128 sub   
10  ¦--Charophyta                   
11  ¦   °--... 2 nodes w/ 459 sub   
12  ¦--Marchantiophyta              
13  ¦   ¦--Jungermanniopsida        
14  ¦   ¦   °--... 3 nodes w/ 76 sub
15  ¦   °--... 2 nodes w/ 104 sub   
16  °--Tracheophyta                 
17      °--... 1 nodes w/ 26 sub    


In [7]:
df_rank <- ToDataFrameTypeCol(plant_families, type = "rank")
df_rank[10:20,] |> tibble::as_tibble() |> print(max_footer_lines = 2)

# A tibble: 11 × 10
   rank_kingdom rank_subkingdom rank_phylum rank_class rank_order  rank_subclass
   <chr>        <chr>           <chr>       <chr>      <chr>       <chr>        
 1 Plantae      NA              Bryophyta   Bryopsida  Bryales     Bryidae      
 2 Plantae      NA              Bryophyta   Bryopsida  Hedwigiales Bryidae      
 3 Plantae      NA              Bryophyta   Bryopsida  Hedwigiales Bryidae      
 4 Plantae      NA              Bryophyta   Bryopsida  Hookeriales Bryidae      
 5 Plantae      NA              Bryophyta   Bryopsida  Hookeriales Bryidae      
 6 Plantae      NA              Bryophyta   Bryopsida  Hookeriales Bryidae      
 7 Plantae      NA              Bryophyta   Bryopsida  Hookeriales Bryidae      
 8 Plantae      NA              Bryophyta   Bryopsida  Hookeriales Bryidae      
 9 Plantae      NA              Bryophyta   Bryopsida  Hypnales    Bryidae      
10 Plantae      NA              Bryophyta   Bryopsida  Hypnales    Bryidae      
11 Plant

## Connect to the database

Read database credentials

In [8]:
if (file.exists("../secrets/database.ini")) {
  tmp <- readLines("../secrets/database.ini")[-1]
  tmp <- strsplit(tmp,'=',fixed=2)
  dbinfo <- unlist(lapply(tmp,function(x) x[2]))
  names(dbinfo) <- unlist(lapply(tmp,function(x) x[1]))
  dbinfo <- data.frame(t(dbinfo),stringsAsFactors=F)
   rm(tmp)
} else {
  cat("No database information found")
}

Connection to the postgresql server (remember to update .pgpass file)

In [9]:
drv <- dbDriver("PostgreSQL") ## 
con <- dbConnect(drv, dbname = dbinfo$database,
                 host = dbinfo$host, port = dbinfo$port,
                 user = dbinfo$user) # this works better in combination with a .pgpass file

## Query database

In [10]:
qry <- 
'SELECT family, genus, "speciesID" as spp, 
"scientificName" as species,
"taxonRank" as taxonrank,
"establishmentMeans" as establishment,
count(distinct q.record_id) as nquadrat, 
count(distinct g8.record_id) as germ8, 
count(distinct r2.record_id) as rect2,
count(distinct g1.record_id) as germ1, 
count(distinct gw1.record_id) as grow1, 
count(distinct r4.record_id) as repr4, 
count(distinct s5.record_id) as surv5, 
count(distinct s6.record_id) as surv6, 
count(distinct s7.record_id) as surv7, 
count(distinct d1.record_id) as disp1, 
count(distinct r3a.record_id) as repr3a, 
count(distinct r3.record_id) as repr3, 
count(distinct s4.record_id) as surv4, 
count(distinct s1.record_id) as surv1
FROM species.caps  
LEFT JOIN form.quadrat_samples q  
 ON "speciesCode_Synonym"=q.species_code::text   
LEFT JOIN litrev.germ8 g8  ON "speciesCode_Synonym"=g8.species_code::text   
LEFT JOIN litrev.rect2 r2  ON "speciesCode_Synonym"=r2.species_code::text   
LEFT JOIN litrev.germ1 g1  ON "speciesCode_Synonym"=g1.species_code::text   
LEFT JOIN litrev.grow1 gw1  ON "speciesCode_Synonym"=gw1.species_code::text   
LEFT JOIN litrev.repr4 r4  ON "speciesCode_Synonym"=r4.species_code::text   
LEFT JOIN litrev.surv5 s5  ON "speciesCode_Synonym"=s5.species_code::text   
LEFT JOIN litrev.surv6 s6  ON "speciesCode_Synonym"=s6.species_code::text   
LEFT JOIN litrev.surv7 s7  ON "speciesCode_Synonym"=s7.species_code::text   
LEFT JOIN litrev.disp1 d1  ON "speciesCode_Synonym"=d1.species_code::text   
LEFT JOIN litrev.repr3a r3a  ON "speciesCode_Synonym"=r3a.species_code::text   
LEFT JOIN litrev.repr3 r3  ON "speciesCode_Synonym"=r3.species_code::text   
LEFT JOIN litrev.surv4 s4  ON "speciesCode_Synonym"=s4.species_code::text   
LEFT JOIN litrev.surv1 s1  ON "speciesCode_Synonym"=s1.species_code::text   

GROUP BY spp'

In [11]:
spp.table<- dbGetQuery(con, qry)
str(spp.table)

'data.frame':	15732 obs. of  20 variables:
 $ family       : chr  "Brassicaceae" "Myrtaceae" "Myrtaceae" "Apiaceae" ...
 $ genus        : chr  "Lepidium" "Eucalyptus" "Melaleuca" "Actinotus" ...
 $ spp          : num  2358 2359 2360 2361 2362 ...
 $ species      : chr  "Lepidium oxytrichum" "Eucalyptus williamsiana" "Melaleuca glomerata" "Actinotus helianthi" ...
 $ taxonrank    : chr  "Species" "Species" "Species" "Species" ...
 $ establishment: chr  "Alive in NSW, Native" "Alive in NSW, Native" "Alive in NSW, Native" "Alive in NSW, Native" ...
 $ nquadrat     : num  0 0 0 0 0 2 1 0 0 0 ...
 $ germ8        : num  0 0 0 0 0 0 0 0 0 0 ...
 $ rect2        : num  0 1 0 2 0 0 0 0 0 0 ...
 $ germ1        : num  0 1 0 1 0 0 0 0 0 1 ...
 $ grow1        : num  0 0 0 0 0 0 0 0 0 0 ...
 $ repr4        : num  0 0 0 1 0 0 0 0 0 0 ...
 $ surv5        : num  0 0 0 2 0 0 0 0 0 0 ...
 $ surv6        : num  0 0 0 0 0 0 0 0 0 0 ...
 $ surv7        : num  0 0 0 1 0 0 0 0 0 0 ...
 $ disp1        : num  3 

In [14]:
with(spp.table, table(taxonrank,establishment))

                   establishment
taxonrank           Alive in NSW, Native Extinct in NSW, Native Hybrid
  Cultivar                             3                      0      0
  Forma                               60                      0      0
  Hybrid                               1                      0      0
  Nothomorph                           0                      0      0
  Species                          10231                     38      0
  Species Hybrid                       9                      0      1
  Subspecies                        1115                      8      0
  Subspecies Hybrid                    8                      0      0
  Variant                              6                      0      0
  Variety                            578                      1      0
                   establishment
taxonrank           Introduced Not Known from NSW
  Cultivar                  12                  0
  Forma                     11                  0
  H

Add column with the plant order from the taxonomy:

In [15]:
spp.table$rank_order <- df_rank$rank_order[match(spp.table$family,df_rank$rank_family)]

In [16]:
spp.table %>% slice(1:5)

family,genus,spp,species,taxonrank,establishment,nquadrat,germ8,rect2,germ1,⋯,repr4,surv5,surv6,surv7,disp1,repr3a,repr3,surv4,surv1,rank_order
<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
Brassicaceae,Lepidium,2358,Lepidium oxytrichum,Species,"Alive in NSW, Native",0,0,0,0,⋯,0,0,0,0,3,0,0,0,1,Brassicales
Myrtaceae,Eucalyptus,2359,Eucalyptus williamsiana,Species,"Alive in NSW, Native",0,0,1,1,⋯,0,0,0,0,0,0,0,0,6,Myrtales
Myrtaceae,Melaleuca,2360,Melaleuca glomerata,Species,"Alive in NSW, Native",0,0,0,0,⋯,0,0,0,0,0,0,0,0,3,Myrtales
Apiaceae,Actinotus,2361,Actinotus helianthi,Species,"Alive in NSW, Native",0,0,2,1,⋯,1,2,0,1,3,0,3,0,14,Apiales
Apiaceae,Apium,2362,Apium prostratum,Species,"Alive in NSW, Native",0,0,0,0,⋯,0,0,0,0,1,0,0,0,1,Apiales


In [17]:
fabales <- c("Fabaceae (Faboideae)", "Fabaceae (Mimosoideae)", "Fabaceae (Caesalpinioideae)" )
unplaced <- c("Dilleniaceae", "Flacourtiaceae")
single_family <- c("Boraginaceae")
rosales <- c("Malaceae")
lamiales <- c("Myoporaceae", "Buddlejaceae")
polypodiales <- c("Pteridaceae")
asparagales <- c("Anthericaceae","Phormiaceae","Lomandraceae", "Agavaceae", "Hyacinthaceae")
ranunculales <- c("Fumariaceae")
malvales <- c("Tiliaceae")


In [18]:
spp.table <- spp.table %>% mutate(
    rank_order = case_when(
            family %in% fabales ~ "Fabales",
            family %in% unplaced ~ "unplaced",
            family %in% rosales ~ "Rosales",
            family %in% lamiales ~ "Lamiales",
            family %in% polypodiales ~ "Polypodiales",
            family %in% asparagales ~ "Asparagales",
            family %in% ranunculales ~ "Ranunculales",
            family %in% malvales ~ "Malvales",
            family %in% single_family ~ sprintf("fam. %s",family),
            is.na(rank_order) ~ "unknown",
            TRUE ~ rank_order
        ))

In [19]:
spp.table %>% 
    filter(rank_order %in% c(NA,"unknown")) %>% 
    group_by(family) %>% 
    summarise(nspp=n()) %>% 
    arrange(desc(nspp))

family,nspp
<chr>,<int>
Unknown Flora,32
Athyriaceae,15
Lemnaceae,13
Viscaceae,13
Stackhousiaceae,12
Characeae,9
Callitrichaceae,8
Dipsacaceae,6
Najadaceae,6


In [20]:
saveRDS(file=here::here('data','Summary-traits-species.rds'),spp.table)

In [21]:
saveRDS(file=here::here('data','NSW-plants-according-to-ALA.rds'),plant_families)

In [22]:
dbDisconnect(con)

[1] TRUE